# Part III – Risk Evaluation and Consistency Testing
This section addresses practical risk validation after model signals are generated and trades are submitted. The focus is on ensuring the system's execution remains aligned with expectations and does not drift materially from the intended portfolio state. While the proposed checklist includes infrastructure-heavy elements suited for production environments, we will focus on those tasks that are feasible and directly relevant to our current live deployment pipeline.

## 11. Cross-checking performance between Zipline and IB 
To evaluate whether the strategy behaves as expected in a live environment, we compare daily and cumulative performance between the model and what was actually executed in the Interactive Brokers account. This includes:

We run the full Zipline Reloaded backtest each day from a fixed start date (2025-08-15) up to the current session(2025-08-18), using updated EOD prices. From this, we extract the model’s NAV and daily returns. Separately, we extract NAV and realized returns from the IB account, based on confirmed trades and reported cash flows.

From both sources, we compute daily return drift (the percent difference between model and live return) and cumulative NAV drift (the divergence in total equity over time). This allows us to catch mismatches due to missed orders, slippage, or rounding effects.

We also compare these results at the individual contract level. For each futures symbol in the portfolio, we track return and NAV drift independently. This helps pinpoint where the largest deviations are coming from—whether from volatility in a single contract, execution errors, or incorrect sizing.

Alongside drift analysis, we generate a performance report that includes key metrics over the tracking period. These include annualized return, cumulative return, volatility, and drawdowns, as well as Sharpe, Sortino, Calmar, and Omega ratios. Additional descriptive stats like skew, kurtosis, tail ratio, and daily VaR are also reported. Finally, alpha and beta to a reference index are included to benchmark relative risk-adjusted performance.

This output serves as a practical diagnostic tool because it ensures that live execution is tracking the model closely enough to maintain confidence in the system, and it highlights when and where manual review is required.

### 11.1. Live NAV Logging from Interactive Brokers

To verify that the strategy behaves as expected in live deployment, we log the daily Net Asset Value (NAV) directly from the Interactive Brokers account. The aim is to compare this value with the NAV produced by the model each day, allowing us to detect divergence due to slippage, fills, fees, or logic mismatch.

Each day after market close, we query two pieces of information from the IB account:

1. NetLiquidation in EUR – the total account value in EUR as the IB account is created in Europe and we cannot choose the base currency.

2. EUR.USD FX Rate – to convert NAV into USD, matching the model's denomination.

Once both values are retrieved, we compute the USD NAV and log it to a CSV file (`ib_nav_log.csv`) along with the timestamp and the FX rate used. If there’s a previous NAV entry, we also calculate the daily percentage change (`drift`). This lets us compare daily returns from the broker with what the model expects.

In [ ]:
import time
import threading
import pandas as pd
from datetime import datetime
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.common import BarData
import os

class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.net_liquidation = None
        self.fx_rate = None
        self.received_nav = threading.Event()
        self.received_fx = threading.Event()

    def accountSummary(self, reqId, account, tag, value, currency):
        if tag == "NetLiquidation" and currency == "EUR":
            self.net_liquidation = float(value)
            print(f"✅ NetLiquidation: {value} {currency}")
            self.received_nav.set()

    def accountSummaryEnd(self, reqId):
        print("✅ Finished account summary.")

    def historicalData(self, reqId: int, bar: BarData):
        self.fx_rate = bar.close
        print(f"✅ FX EUR.USD Rate: {bar.close}")
        self.received_fx.set()

    def historicalDataEnd(self, reqId: int, start: str, end: str):
        print("✅ Finished receiving FX data.")

def create_fx_contract(pair_str):
    base, quote = pair_str.split(".")
    contract = Contract()
    contract.symbol = base
    contract.secType = "CASH"
    contract.exchange = "IDEALPRO"
    contract.currency = quote
    return contract

# === Start IB connection ===
app = IBapi()
app.connect("127.0.0.1", 7497, clientId=101)
threading.Thread(target=app.run, daemon=True).start()
time.sleep(1)

# === Request NAV (EUR) ===
app.reqAccountSummary(9001, "All", "NetLiquidation")
if not app.received_nav.wait(timeout=10):
    print("❌ NAV not received within timeout.")
    exit()

# === Request FX EUR.USD ===
fx_contract = create_fx_contract("EUR.USD")
app.reqHistoricalData(
    reqId=5001,
    contract=fx_contract,
    endDateTime="",
    durationStr="1 D",
    barSizeSetting="1 day",
    whatToShow="MIDPOINT",
    useRTH=0,
    formatDate=2,
    keepUpToDate=False,
    chartOptions=[]
)
if not app.received_fx.wait(timeout=10):
    print("❌ FX rate not received within timeout.")
    exit()

# === Compute NAV in USD ===
nav_eur = app.net_liquidation
fx = app.fx_rate
nav_usd = nav_eur * fx
today = pd.Timestamp.now(tz='UTC').strftime('%Y-%m-%d %H:%M:%S')

print(f"\n💰 NAV in EUR: {nav_eur:,.2f}")
print(f"💱 EUR.USD FX: {fx:.6f}")
print(f"💵 NAV in USD: {nav_usd:,.2f}")

# === Append to CSV file with drift ===
out_path = "ib_nav_log.csv"
new_row = {
    "Date": today,
    "NAV_EUR": nav_eur,
    "FX_EURUSD": fx,
    "NAV_USD": nav_usd,
    "NAV_USD_Drift_Pct": None  # will compute next
}

if os.path.exists(out_path):
    df = pd.read_csv(out_path)
    if not df.empty:
        previous_usd = df.iloc[-1]["NAV_USD"]
        if previous_usd != 0:
            drift = ((nav_usd - previous_usd) / previous_usd) * 100
            new_row["NAV_USD_Drift_Pct"] = round(drift, 4)
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
else:
    df = pd.DataFrame([new_row])

df.to_csv(out_path, index=False)
print(f"\n✅ NAV logged to {out_path}")

# === Print drift comparison ===
if new_row["NAV_USD_Drift_Pct"] is not None:
    print(f"\n📈 NAV Drift vs Previous: {new_row['NAV_USD_Drift_Pct']:.2f}%")
else:
    print("\nℹ️ No prior NAV to compare.")

In [ ]:
app.disconnect()

Once the IB NAV is retrieved, the model NAV from the Zipline backtest is aligned with the live NAV reported by IB. The purpose is to measure deviations and check consistency between the backtest and the brokerage account.
We begin with daily percentage returns from the backtest. Starting from $1,000,000 initial capital, the cumulative NAV is computed by compounding these returns. Only dates common to both datasets are kept, so weekends and non-trading days are excluded naturally.

For each source we calculate day-over-day percentage changes:

- Zipline_Model_Change = model NAV return
- IB_Model_Change = actual account NAV return
- Drift_Return_Pct = IB return – Zipline return

This drift shows whether live performance diverges from the model.

The results are saved in nav_drift_comparison.csv, which can be used to flag outliers and monitor where differences between the model and live account accumulate.

In [ ]:
import pandas as pd

# === File Paths ===
zipline_path = r"C:\Users\Juanan\Downloads\CQF\1.Futures_Trend_Following\1.Zipline_Trades\2.Deployment_Backtest\returns_2025-08-15_to_2025-08-18_TF.csv"
ib_path = r"C:\Users\Juanan\Downloads\CQF\ib_nav_log.csv"
output_path = r"C:\Users\Juanan\Downloads\CQF\nav_drift_comparison.csv"

# === Load Zipline returns ===
zip_df = pd.read_csv(zipline_path)
zip_df["Date"] = pd.to_datetime(zip_df.iloc[:, 0]).dt.date
zip_df = zip_df.rename(columns={"returns": "Zipline_Model_Change"})
zip_df = zip_df.sort_values("Date")

# Reconstruct NAV from initial capital
initial_nav = 1_000_000
zip_df["Zipline_Model_NAV"] = initial_nav * (1 + zip_df["Zipline_Model_Change"]).cumprod()

# === Load IB NAV ===
ib_df = pd.read_csv(ib_path)
ib_df["Date"] = pd.to_datetime(ib_df["Date"]).dt.date
ib_df = ib_df.rename(columns={"NAV_USD": "IB_Model_NAV"})

# === Merge ===
merged = pd.merge(zip_df[["Date", "Zipline_Model_NAV"]], ib_df[["Date", "IB_Model_NAV"]], on="Date", how="inner")

# === Calculate changes directly from NAVs ===
merged["Zipline_Model_Change"] = merged["Zipline_Model_NAV"].pct_change()
merged["IB_Model_Change"] = merged["IB_Model_NAV"].pct_change()
merged["Drift_Return_Pct"] = merged["IB_Model_Change"] - merged["Zipline_Model_Change"]

# === Convert percentage returns to 0.00% format ===
merged["Zipline_Model_Change"] = (merged["Zipline_Model_Change"] * 100).round(4)
merged["IB_Model_Change"] = (merged["IB_Model_Change"] * 100).round(4)
merged["Drift_Return_Pct"] = (merged["Drift_Return_Pct"] * 100).round(4)

# === Format NAV columns with commas ===
merged["Zipline_Model_NAV"] = merged["Zipline_Model_NAV"].map("{:,.2f}".format)
merged["IB_Model_NAV"] = merged["IB_Model_NAV"].map("{:,.2f}".format)

# === Final output ===
output = merged[[
    "Date",
    "Zipline_Model_NAV",
    "IB_Model_NAV",
    "Zipline_Model_Change",
    "IB_Model_Change",
    "Drift_Return_Pct"
]]

# === Save and display ===
output.to_csv(output_path, index=False)
display(output)
print(f"\n✅ Drift comparison saved to: {output_path}")

Note that discrepancies are larger because the initial capital in IB is higher due to past operations in the demo account, and it cannot be reset in the personal account. For this reason, the visualization should be considered only as an example.

This section then plots the daily return series from both the model and the IB account, together with the drift. The plot helps to spot deviations, execution errors, or data mismatches. A consistent strategy with proper execution should show small and infrequent drift, while systematic or growing divergence would point to potential issues.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# === Load comparison file ===
comparison_path = r"C:\Users\Juanan\Downloads\CQF\nav_drift_comparison.csv"
df = pd.read_csv(comparison_path)

# === Parse date ===
df["Date"] = pd.to_datetime(df["Date"])

# === Convert percentage columns to numeric ===
for col in ["Zipline_Model_Change", "IB_Model_Change", "Drift_Return_Pct"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# === Plot: Daily Return Comparison ===
plt.figure(figsize=(12, 6))
plt.plot(df["Date"], df["Zipline_Model_Change"], label="Zipline Model Change (%)", marker="o")
plt.plot(df["Date"], df["IB_Model_Change"], label="IB Model Change (%)", marker="o")
plt.plot(df["Date"], df["Drift_Return_Pct"], label="Drift Return (%)", linestyle="--", color="red")
plt.title("Daily Return Comparison")
plt.xlabel("Date")
plt.ylabel("Daily Return (%)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

Having aligned both NAV series and visualized the daily tracking error, the next step is to quantify how well the live portfolio matches the model over time. We compute performance metrics for both the Zipline model and the IB live account using the daily return series we already have. Metrics include cumulative and annualized return, volatility, Sharpe, Sortino, drawdowns, and ratios that describe the shape and asymmetry of returns. These are standard in assessing whether the live portfolio is tracking risk and return consistently.

Finally, we regress IB returns on the model to extract alpha and beta — not for predictive use, but to assess if the live account has consistent exposure to the model and whether there’s any persistent drift or unexplained return component.

In [ ]:
import pandas as pd
import numpy as np

# === Load data ===
path = r"C:\Users\Juanan\Downloads\CQF\nav_drift_comparison.csv"
df = pd.read_csv(path)
df["Date"] = pd.to_datetime(df["Date"])

# === Convert returns to decimal ===
df["Zipline_Model_Change"] = pd.to_numeric(df["Zipline_Model_Change"], errors="coerce") / 100
df["IB_Model_Change"] = pd.to_numeric(df["IB_Model_Change"], errors="coerce") / 100

# === Drop NaNs ===
df = df.dropna(subset=["Zipline_Model_Change", "IB_Model_Change"])

# === Settings ===
periods_per_year = 252

def compute_metrics(returns: pd.Series, benchmark: pd.Series = None, name=""):
    cumulative = (1 + returns).prod() - 1
    annualized = (1 + cumulative) ** (periods_per_year / len(returns)) - 1
    volatility = returns.std() * np.sqrt(periods_per_year)
    sharpe = annualized / volatility if volatility > 0 else np.nan
    downside_std = returns[returns < 0].std() * np.sqrt(periods_per_year)
    sortino = annualized / downside_std if downside_std > 0 else np.nan
    drawdown = (1 + returns).cumprod().div((1 + returns).cumprod().cummax()) - 1
    max_dd = drawdown.min()
    calmar = annualized / abs(max_dd) if max_dd != 0 else np.nan
    omega = returns[returns > 0].mean() / abs(returns[returns < 0].mean()) if not returns[returns < 0].empty else np.nan
    skewness = returns.skew()
    kurt = returns.kurt()
    tail_ratio = returns[returns > 0].quantile(0.95) / abs(returns[returns < 0].quantile(0.05)) if not returns[returns < 0].empty else np.nan
    var_95 = returns.quantile(0.05)

    alpha = beta = np.nan
    if benchmark is not None and not benchmark.empty:
        cov = np.cov(returns, benchmark)[0][1]
        var_bench = np.var(benchmark)
        beta = cov / var_bench if var_bench != 0 else np.nan
        alpha = (returns.mean() - beta * benchmark.mean()) * periods_per_year if not np.isnan(beta) else np.nan

    return {
        "Name": name,
        "Cumulative Return (%)": round(cumulative * 100, 2),
        "Annualized Return (%)": round(annualized * 100, 2),
        "Volatility (%)": round(volatility * 100, 2),
        "Sharpe Ratio": round(sharpe, 3),
        "Sortino Ratio": round(sortino, 3),
        "Max Drawdown (%)": round(max_dd * 100, 2),
        "Calmar Ratio": round(calmar, 3),
        "Omega Ratio": round(omega, 3),
        "Skewness": round(skewness, 3),
        "Kurtosis": round(kurt, 3),
        "Tail Ratio": round(tail_ratio, 3),
        "Daily VaR (5%) (%)": round(var_95 * 100, 2),
        "Alpha vs Zipline (%)": round(alpha * 100, 2) if name == "IB" else None,
        "Beta vs Zipline": round(beta, 4) if name == "IB" else None
    }

# === Compute metrics ===
zip_metrics = compute_metrics(df["Zipline_Model_Change"], name="Zipline")
ib_metrics = compute_metrics(df["IB_Model_Change"], benchmark=df["Zipline_Model_Change"], name="IB")

# === Combine ===
metrics_df = pd.DataFrame([zip_metrics, ib_metrics])
display(metrics_df)

Right now we only have one interval to compare the Zipline NAV and the IB NAV. That makes the drift number and metrics basically meaningless on its own — it could be just noise from rounding, FX rates, or a timing mismatch. The only way to judge whether there is a systematic difference is to collect more data over time. Once we have a longer series, we can see whether the drift averages out or whether it compounds in one direction. Until then, it’s just a single observation, not something we can act on.

### 11.2. Position-Level Attribution – Open Positions and Cumulative Returns

Following the portfolio-level drift and performance consistency checks, we now move to a position-level analysis. The goal here is to identify which contracts are currently open as of the most recent trading date, trace back to their entry points, and compute the return each position has generated since entry. This gives us a clearer view of which exposures are contributing to current performance and where risk is currently concentrated.

The script loads the full position history from the Zipline reloaded backtest, where each column represents a contract's USD value over time. After identifying the most recent trading day, we isolate contracts with non-zero positions — these are the trades still open.

For each open contract, we locate the most recent day it transitioned from zero to a non-zero USD value, assuming that marks the entry point. We then compute the cumulative return as the percentage change in USD value from entry to the current date.

The output includes:

- Contract name

- Entry date

- Last date

- Entry and current USD values

- Cumulative return %

Cash positions are excluded. The final table is sorted by return in descending order to highlight the top contributors.


In [ ]:
import pandas as pd

# === Load positions file ===
positions_file = r"C:\Users\Juanan\Downloads\CQF\1.Futures_Trend_Following\1.Zipline_Trades\2.Deployment_Backtest\positions_2025-08-15_to_2025-08-18_TF.csv"
df = pd.read_csv(positions_file)

# === Fix date column ===
df["Date"] = pd.to_datetime(df["index"])
df = df.drop(columns=["index"])

# === Identify last date (most recent) ===
last_date = df["Date"].max()
last_positions = df[df["Date"] == last_date].drop(columns=["Date"])

# === Find contracts with open positions on the last date (non-zero) ===
open_contracts = last_positions.loc[:, (last_positions != 0).any(axis=0)].T
open_contracts.columns = ["Open_Position_USD"]
open_contracts["Contract"] = open_contracts.index

# === Initialize return tracking ===
returns = []

# === Loop through each open contract ===
for contract in open_contracts["Contract"]:
    contract_series = df[["Date", contract]].copy()
    contract_series = contract_series[contract_series[contract] != 0]

    entry_date = contract_series["Date"].iloc[-1]
    last_value = contract_series[contract_series["Date"] == last_date][contract].values[0]
    entry_value = contract_series[contract_series["Date"] == entry_date][contract].values[0]

    cumulative_return = ((last_value - entry_value) / abs(entry_value)) * 100 if entry_value != 0 else 0

    returns.append({
        "Contract": contract,
        "Entry_Date": entry_date.date(),
        "Last_Date": last_date.date(),
        "Entry_Value_USD": entry_value,
        "Last_Value_USD": last_value,
        "Cumulative_Return_%": round(cumulative_return, 4)
    })

# === Create final DataFrame ===
returns_df = pd.DataFrame(returns)

# === Format values ===
returns_df["Entry_Value_USD"] = returns_df["Entry_Value_USD"].map(lambda x: f"{x:,.2f}")
returns_df["Last_Value_USD"] = returns_df["Last_Value_USD"].map(lambda x: f"{x:,.2f}")

# === Optional: drop cash row ===
returns_df = returns_df[returns_df["Contract"].str.lower() != "cash"]

# === Sort by return descending ===
returns_df = returns_df.sort_values(by="Cumulative_Return_%", ascending=False)

# === Display full ===
pd.set_option("display.max_rows", None)
display(returns_df)

Having reviewed open positions from the backtest, we now cross-check with the live account via the IB API. This ensures that reported exposures match what is actually held at the broker.

In [ ]:
import time
import threading
from decimal import Decimal
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import pandas as pd
from datetime import datetime
import os
import sys

class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.positions = []
        self.done = threading.Event()

    def position(self, account, contract: Contract, position, avgCost):
        try:
            if getattr(contract, "secType", None) == "FUT" and position != 0:
                # Normalize numeric types
                pos_dec = Decimal(str(position))
                avg_dec = avgCost if isinstance(avgCost, Decimal) else Decimal(str(avgCost))
                entry_val = pos_dec * avg_dec

                self.positions.append({
                    "Contract": f"Future({getattr(contract, 'conId', 'NA')} [{getattr(contract, 'localSymbol', 'NA')}])",
                    "Entry_Date": "N/A",
                    "Last_Date": datetime.today().strftime('%Y-%m-%d'),
                    "Entry_Value_USD": float(entry_val),
                    "Last_Value_USD": float(entry_val),
                    "Cumulative_Return_%": 0.0
                })
        except Exception as e:
            print(f"⚠️ position() error for {getattr(contract,'localSymbol','?')}: {e}", file=sys.stderr)

    def positionEnd(self):
        print("✅ Finished receiving open positions.")
        self.done.set()

# === Run IB API ===
app = IBapi()
app.connect("127.0.0.1", 7497, clientId=102)
thread = threading.Thread(target=app.run, daemon=True)
thread.start()

# === Request positions ===
time.sleep(1)
app.reqPositions()
if not app.done.wait(timeout=15):
    print("❌ Timeout waiting for position data.")
    app.disconnect()
    sys.exit(1)

# === Create DataFrame and export ===
df = pd.DataFrame(app.positions)

if df.empty:
    print("ℹ️ No open futures positions returned by IB.")
else:
    for col in ("Entry_Value_USD", "Last_Value_USD", "Cumulative_Return_%"):
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce").round(2)

    # Save directly to Downloads\CQF
    output_path = r"C:\Users\Juanan\Downloads\CQF\ib_open_positions.csv"
    df.to_csv(output_path, index=False)
    print(f"\n✅ Saved open futures positions to: {output_path}\n")

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)

display(df)


# === Clean up ===
app.disconnect()


The difference between the two outputs arises because Interactive Brokers only reports positions that are already filled. Orders that are submitted but not executed — for example, because the market was closed or the limit price was not reached — do not appear in the position list. Once executed, the position will appear. If the market moves away from the limit price, the order remains pending until adjusted or canceled.
This closes the loop between backtest and the live account. If the strategy logic is consistent and market structure unchanged, the two views should align over time. Any differences can then be measured directly instead of assumed.

As a final note, full containerization of the execution pipeline will not be implemented at this stage. The system depends on Interactive Brokers’ TWS/Gateway for API connectivity, which does not run reliably in headless environments like Docker without workarounds. In addition, Zipline Reloaded requires daily ingestion of the full Norgate data bundle, which makes container orchestration inefficient.

Instead, the pipeline is modularized into scripts for ingestion, backtest, signal extraction, and order generation. These can be scheduled individually with a task scheduler or run manually after market close to maintain control and avoid unintended live order flow.

For data storage, we are working directly with daily-exported bundles from Norgate. While this approach is simple and sufficient for development and execution, it has limitations: sensitivity to schema changes. A longer-term plan is to introduce a dedicated securities database (e.g., MySQL or PostgreSQL). This would decouple ingestion from model execution, simplify metadata handling (sector tags, tick sizes, contract specifications), and provide consistent access across tools. For now, given the limited scope of assets and data, the current approach remains acceptable and easy to inspect during the buildout phase.